In [71]:
%%bash
# Predictors (Variants in interest)
cat predictor/_pos*.txt | LANG=C sort | uniq |sed '1 iPOS' > predictor/select.txt
awk -F' ' '/rs/{print $1,$2,$3}' predictor/_snp* | LANG=C sort | sed '1 iPOS ID NearGenpoe' |\
    sed 's/ /\t/g' | uniq -w 15 > predictor/snpInfo.txt
awk -F' ' '{print $1,toupper($4),$7}' predictor/Meta5.tab | sed 's/ /\t/g' | sed 's/chr//g' > predictor/grs.txt

In [27]:
%%bash
# create the data with individual (row) /snps (column) for selected SNPs in interst (R>0.3, maf>0.01)
# Bring already created file (maf01rsq3)
zless /data/LNG/CORNELIS_TEMP/progression_GWAS/PARKWEST/chr22.dose.vcf.gz | head -n 20| cut -f1-5
module load vcftools
zcat /data/LNG/CORNELIS_TEMP/progression_GWAS/PARKWEST/chr22.dose.vcf.gz | vcf-to-tab  >out.tab

##fileformat=VCFv4.1
##filedate=2018.5.22
##source=Minimac3
##contig=<ID=22>
##FILTER=<ID=GENOTYPED,Description="Marker was genotyped AND imputed">
##FILTER=<ID=GENOTYPED_ONLY,Description="Marker was genotyped but NOT imputed">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=DS,Number=1,Type=Float,Description="Estimated Alternate Allele Dosage : [P(0/1)+2*P(1/1)]">
##FORMAT=<ID=GP,Number=3,Type=Float,Description="Estimated Posterior Probabilities for Genotypes 0/0, 0/1 and 1/1 ">
##INFO=<ID=AF,Number=1,Type=Float,Description="Estimated Alternate Allele Frequency">
##INFO=<ID=MAF,Number=1,Type=Float,Description="Estimated Minor Allele Frequency">
##INFO=<ID=R2,Number=1,Type=Float,Description="Estimated Imputation Accuracy">
##INFO=<ID=ER2,Number=1,Type=Float,Description="Empirical (Leave-One-Out) R-square (available only for genotyped variants)">
#CHROM	POS	ID	REF	ALT
22	16050435	22:16050435	T	C
22	16050783	22:16050783	A	G
22	16050822	22:16050822	G	A
22	1605092

[+] Loading samtools 1.9  ... 
[+] Loading vcftools  0.1.15 
Leading or trailing space in attr_key-attr_value pairs is discouraged:
	[Description] [Estimated Posterior Probabilities for Genotypes 0/0, 0/1 and 1/1 ]
	FORMAT=<ID=GP,Number=3,Type=Float,Description="Estimated Posterior Probabilities for Genotypes 0/0, 0/1 and 1/1 ">


In [25]:
head out.tab

SyntaxError: invalid syntax (<ipython-input-25-ced9d2bddc60>, line 1)

In [36]:
%%bash
# Create Phenotype files

FOLDER=pheno
mkdir -p $FOLDER
cd $FOLDER
for DATASET in $(ls ../../dataset/);do
#     mkdir -p $DATASET
#     cd $DATASET
    mkdir -p lgsbl
    mkdir -p coxhm
    mkdir -p lnmxs
    mkdir -p lncns
    mkdir -p lnmxi
    mkdir -p lnsgl
done


# 1    Need to match phenotype ID and IID.
# 1.1    Extract IID from imputed file
rm t/_IDlist_imputed.txt
for DATASET in $(ls ../dataset);do
  cat /data/LNG/CORNELIS_TEMP/progression_GWAS/$DATASET/plink_files_hard/*.fam | sed "s/^/$DATASET:/g" | cut -f1 | cut -d " " -f1 | sed 's/:/'$'\t/g' >> t/_IDlist_imputed.txt
done
# 1.2    Creating HBS ID matching file. (No formal table so compare genotyping file)
module load plink
GNTYP_IN=/data/LNG/CORNELIS_TEMP/progression_GWAS/
plink --bfile data/HBS_original/pruned --bmerge $GNTYP_IN/pre_impute_data/HBS --make-bed --out t/V1
plink --bfile data/HBS_original/pruned --exclude t/V1-merge.missnp --make-bed --out t/V2
plink --bfile t/V2 --bmerge $GNTYP_IN/pre_impute_data/HBS --make-bed --out t/V3
plink --bfile t/V3 --genome --min 0.7 --out t/V4
mv t/V4.genome data/HBSIDmatch.txt

In [1]:
%%bash
# 1.3    
module load R
Rscript --vanilla CreatePhenoFile.R

find IID without ID. No output is good 
# A tibble: 0 x 4
# Groups:   DATASET, IID [0]
# ... with 4 variables: DATASET <chr>, IID <chr>, ID <chr>, COHORT <chr>
complete step1 
compete step2 
CORIELL    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
DATATOP    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
DIGPD_chip    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
DIGPD_neuroX    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
HBS    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
OSLO    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
PARKFIT    lgsbl    lnsgl    lnmxi/lnmxs    lncns    coxhm 
PARKWEST    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
PDBP    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
PICNICS    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
PPMI    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
PRECEPT    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 
SCOPA    lgsbl    NO_lnsgl    lnmxi/lnmxs    lncns    coxhm 

[-] Unloading gcc  7.2.0  ... 
[-] Unloading GSL 2.4 for GCC 7.2.0 ... 
[-] Unloading openmpi 3.0.0  for GCC 7.2.0 
[-] Unloading R 3.5.0_build2 
[+] Loading gcc  7.2.0  ... 
[+] Loading GSL 2.4 for GCC 7.2.0 ... 
[+] Loading openmpi 3.0.0  for GCC 7.2.0 
[+] Loading R 3.5.0_build2 
Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: data.table

Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



In [4]:
%%bash
# Create output folders
for FOLDER in output outGWAS outGRS;do
    mkdir -p $FOLDER
    cd $FOLDER
    for DATASET in $(ls ../../dataset/);do
        mkdir -p $DATASET
        cd $DATASET
        mkdir -p lgsbl
        mkdir -p coxhm
        mkdir -p lnmxs
        mkdir -p lncns
        mkdir -p lnmxi
        mkdir -p lnsgl
        cd ..
    done
    cd ..
done

In [46]:
%%bash
# RVTESTS for logistic regression an linear regression at baseline
rm -f t/swarm_rvtest*
GNTYP_IN="/data/LNG/CORNELIS_TEMP/progression_GWAS"
for MODEL in "lgsbl" "lnsgl";do
    for i in $(grep $MODEL models.txt);do 
        DATASET=$(echo $i | cut -d ";" -f2)
        OUTCOME=$(echo $i | cut -d ";" -f3)
        COVS=$(echo $i | cut -d ";" -f4)
        mkdir -p outGWAS/$DATASET/$MODEL/$OUTCOME
        for CHNUM in {1..22};do
            echo "
rvtest --noweb --hide-covar --rangeFile ../dataset/$DATASET/maf01rsq3.info \
    --inVcf $GNTYP_IN/$DATASET/chr$CHNUM.dose.vcf.gz \
    --pheno pheno/$MODEL/$DATASET.txt --pheno-name $OUTCOME \
    --covar pheno/$MODEL/"$DATASET"_cov.txt --covar-name "$COVS",PC1,PC2,PC3,PC4,PC5 \
    --out outGWAS/$DATASET/$MODEL/$OUTCOME/chr$CHNUM --single wald; \
echo \"$CHNUM;$i\" >>t/swarm_rvtest_log.txt" >> t/swarm_rvtest.txt
        done
        echo "
cat outGWAS/$DATASET/$MODEL/$OUTCOME/chr*.SingleWald.assoc | awk '{print \$1\":\"\$2,\$4,\$3,\$7,\$8,\$9}'|\
 grep -v ^CHROM | LANG=C sort | sed '1 iPOS A1 A2 BETA SE P' | sed 's/ /\t/g' |\
 join --header  -t\$'\t' - ../dataset/$DATASET/maf01rsq3_sorted.txt | cut -f 1-6,9,10\
 > outGWAS/$DATASET/$MODEL/$OUTCOME/all.tab" >> t/swarm_rvtest_after.txt
    done
done
echo "swarm -f t/swarm_rvtest.txt --time=6:00:00 -p 2 -b 4 --logdir t/swarm_rvtest --module rvtests --devel"
echo "swarm -f t/swarm_rvtest_after.txt --time=0:10:00 -p 2 -b 16 --logdir t/swarm_rvtest --partition=quick --devel"
echo "job1=\$(swarm -f t/swarm_rvtest.txt --time=6:00:00 -p 2 -b 4 --logdir t/swarm_rvtest --module rvtests)"
echo "swarm -f t/swarm_rvtest_after.txt --time=0:10:00 -p 2 -b 16 --logdir t/swarm_rvtest --partition=quick --dependency=afterany:\$job1"

swarm -f t/swarm_rvtest.txt --time=6:00:00 -p 2 -b 4 --logdir t/swarm_rvtest --module rvtests --devel
swarm -f t/swarm_rvtest_after.txt --time=0:10:00 -p 2 -b 16 --logdir t/swarm_rvtest --partition=quick --devel
job1=$(swarm -f t/swarm_rvtest.txt --time=6:00:00 -p 2 -b 4 --logdir t/swarm_rvtest --module rvtests)
swarm -f t/swarm_rvtest_after.txt --time=0:10:00 -p 2 -b 16 --logdir t/swarm_rvtest --partition=quick --dependency=afterany:$job1


In [47]:
%%bash
# job1=$(swarm -f t/swarm_rvtest.txt --time=6:00:00 -p 2 -b 4 --logdir t/swarm_rvtest --module rvtests)
# swarm -f t/swarm_rvtest_after.txt --time=0:10:00 -p 2 -b 16 --logdir t/swarm_rvtest --partition=quick --dependency=afterany:$job1
# echo $job1

9664803
9664799


In [ ]:
%%bash
# For genetic risk score, use R for the analysis. 
echo '
library(dplyr);library(data.table)
DATASETs=list.files("../dataset/")
DATASET=DATASETs[1]
SCOREs=list.files(paste("../dataset", DATASET, "grs", sep="/"),)
'

In [2]:
%%bash 
# Cohort level longitudinal analysis with R (coxhm, lnmxs and lnmxi)

## coxhm 
# args=c("coxhm;CORIELL;DEMENTIA;FEMALE,YEARSEDUC,FAMILY_HISTORY,AAO,BLDfDIAG,DOPA,AGONIST", "../dataset/CORIELL/maf01rsq3_20Kcut/cut22.2.txt.gz", "outGWAS")
echo '
args <- commandArgs(trailingOnly = TRUE)
# Variables
KEY=strsplit(args[1], ";")[[1]]
ANALYSIS=KEY[1]
DATASET=KEY[2]
OUTCOME=KEY[3]
COVs   =KEY[4]
COVPC  =paste(gsub(",", "+", COVs), "+PC1+PC2+PC3+PC4+PC5", sep="")
INFILE  = paste("gzcat -f", args[2], sep=" ")
OUT = args[3]
library(data.table);library(dplyr)
## Read genptypes
SNPset = fread(INFILE)
SNPs = names(SNPset)[-c(1:2)] # 1 IID, DOSE SNP name starts from col 3
# Read phenotypes
cohort=fread(paste("pheno/", ANALYSIS, "/", OUTCOME, "_", DATASET, ".txt", sep = "")) %>% arrange(IID, TSTART) %>%
  select(c("IID", "TSTART", "TSTOP", OUTCOME, strsplit(COVPC, "\\+")[[1]])) %>% filter(complete.cases(.))
# rescaling
cohort[,(strsplit(COVPC, "\\+")[[1]])] = as.data.frame(scale(cohort[,(strsplit(COVPC, "\\+")[[1]])]))
## Merge data
cohort_snp = left_join(cohort, SNPset, by = "IID")
# ANALYSIS
library(survival)
cohort_snp$SurvObj1 = with(cohort_snp, Surv(TSTART, TSTOP, OUTCOME == 1))
test.listfunc = function(x){
  MODEL = paste("SurvObj1 ~", "`", SNPs[x], "`+", COVPC, sep = "")
  testCox = try(coxph(eval(parse(text = MODEL)), data = cohort_snp),silent = T)
  if(class(testCox)[1]=="try-error"){
    sumstat=c(SNPs[x], "NoConverge", rep(NA,4))
  }else{
    temp= summary(testCox)$coefficients
    if(grep(substr(SNPs[x],1,7), rownames(temp)) %>% length == 0){ # In this case, SNP is dropeed from the model
      sumstat=c(SNPs[x], "NoVforSNP", rep(NA, 4))
    }else{
      RES = temp[1,]
      EVENT_OBS = paste(testCox$nevent, testCox$n, sep="_")
      sumstat <- c(SNPs[x], EVENT_OBS, as.numeric(RES[4]), RES[1], RES[3], RES[5])
    }
  }
  return(sumstat)
}
temp = lapply(1:length(SNPs), test.listfunc)
temp2 = do.call(rbind, temp) %>% data.frame%>%filter(complete.cases(.))
names(temp2)=c("POS_A2_A1", "EVENT_OBS", "Tvalue", "BETA", "SE", "P")
NEWDIR = paste(OUT, DATASET, ANALYSIS, OUTCOME, sep = "/")
dir.create(NEWDIR, recursive = T, showWarnings = F)
FILENAME = strsplit(INFILE, "/")[[1]][5] %>% substr(.,1,nchar(.)-3) %>% sub("\\.", "_", .) # cut22_2.txt
write.table(temp2, paste(NEWDIR, FILENAME, sep="/"), row.names = F, quote = F, sep = "\t")
line=paste("Rscript --vanilla t/", ANALYSIS, ".R ", paste(args, collapse = " "), sep="")
write(line,file=paste("finish_GWAS_", ANALYSIS, ".txt", sep=""),append=TRUE)
' > t/coxhm.R

# lnmxi
# args=c("lnmxi;CORIELL;SEADL;FEMALE,YEARSEDUC,FAMILY_HISTORY,AAO,YEARfDIAG,DOPA,AGONIST", "../dataset/CORIELL/maf01rsq3_20Kcut/cut22.2.txt.gz", "outGWAS")
echo '
# Read phenotype
library(lme4)
cohort=fread(paste("pheno/", ANALYSIS, "/", DATASET, ".txt", sep = "")) %>% arrange(IID, YEARfDIAG) %>% 
  select(c("IID", OUTCOME, strsplit(COVPC, "\\+")[[1]])) %>% filter(complete.cases(.))
# rescaling
cohort[,(strsplit(COVPC, "\\+")[[1]])] = as.data.frame(scale(cohort[,(strsplit(COVPC, "\\+")[[1]])]))
# Merge data
cohort_snp = left_join(cohort, SNPset, by = "IID")
# ANALYSIS
test.listfunc = function(x){
  # Models
  MODEL = paste(OUTCOME, "~", "`", SNPs[x], "`+", paste(COVPC, collapse="+"), "+(1|IID)", sep = "")
  testLmer = try(lmer(eval(parse(text = MODEL)), data = cohort_snp),silent = T)
  if(class(testLmer)[1]=="try-error"){
    sumstat=c(SNPs[x], "NoConverge", rep(NA,4))
  }else{
    temp = summary(testLmer)
    temp1 = temp$coefficients
    if(grep(substr(SNPs[x],1,7), rownames(temp1)) %>% length == 0){ # In this case, SNP is dropeed from the model
      sumstat=c(SNPs[x], "Dropped", rep(NA,4))
    }else{
      RES = temp1[2,] # The first row is intercept
      PV_APPROX = 2 * pnorm(-abs(RES[3])) # df is enough large for approximation
      OBS_N = paste(length(temp$residuals), "_", temp$ngrps, sep="")
      sumstat <- c(SNPs[x], OBS_N, RES[3], RES[1], RES[2], PV_APPROX)
    }
  }
  return(sumstat)
}' > t/model_body.txt
sed -n 1,15p t/coxhm.R > t/model_head.txt
sed -n 43,49p t/coxhm.R > t/model_tail.txt
cat t/model_head.txt t/model_body.txt t/model_tail.txt > t/lnmxi.R

# lnmxs  
#fixed slope model. (Randome slope cannnot be estimated in some cohorts)
# args=c("lnmxs;CORIELL;UPDRS4_scaled;FEMALE,YEARSEDUC,FAMILY_HISTORY,AAO,YEARfDIAG,DOPA,AGONIST", "../dataset/CORIELL/maf01rsq3_20Kcut/cut22.2.txt.gz", "outGWAS")
echo '
test.listfunc = function(x){
  # Models
  MODEL = paste(OUTCOME, "~", "`", SNPs[x], "`+ `", SNPs[x], "`*YEARfDIAG + ", paste(COVPC, collapse="+"), "+(1|IID)", sep = "")
  testLmer = try(lmer(eval(parse(text = MODEL)), data = cohort_snp),silent = T)
  if(class(testLmer)[1]=="try-error"){
    sumstat=NA
  }else{
    temp = summary(testLmer)
    temp1 = temp$coefficients %>% data.frame
    names(temp1)=c("BETA", "SE", "Tvalue")
    sumstat=temp1[grep(substr(SNPs[x],1,7), rownames(temp1)),]
    if(nrow(sumstat)==0){
      sumstat=NA
    }else{
        sumstat$POS_A2_A1=rownames(sumstat)
        sumstat = sumstat %>% 
        mutate(P= 2 * pnorm(-abs(Tvalue)), 
            OBS_N=paste(length(temp$residuals), "_", temp$ngrps, sep=""),
            POS_A2_A1=gsub("`", "", POS_A2_A1)) %>% 
        select("POS_A2_A1", "OBS_N", "Tvalue", "BETA", "SE", "P")
    }
  }
  return(sumstat)
}
temp = lapply(1:length(SNPs), test.listfunc)
temp2 = do.call(rbind, temp) %>% data.frame %>% filter(complete.cases(.))
NEWDIR = paste(OUT, DATASET, ANALYSIS, OUTCOME, sep = "/")
NEWDIR2 = paste(OUT, DATASET, paste(ANALYSIS, "intcp", sep = "_"), OUTCOME, sep = "/")
dir.create(NEWDIR, recursive = T, showWarnings = F)
dir.create(NEWDIR2, recursive = T, showWarnings = F)
FILENAME = strsplit(INFILE, "/")[[1]][5] %>% substr(.,1,nchar(.)-3) %>% sub("\\.", "_", .) # cut22_2.txt
temp2 %>% filter(grepl(":YEARfDIAG", POS_A2_A1)) %>% 
  mutate(POS_A2_A1=sub(":YEARfDIAG", "", POS_A2_A1)) %>%
  write.table(., paste(NEWDIR, FILENAME, sep="/"), row.names = F, quote = F, sep = "\t")
temp2 %>% filter(!grepl(":YEARfDIAG", POS_A2_A1)) %>% 
  write.table(., paste(NEWDIR2, FILENAME, sep="/"), row.names = F, quote = F, sep = "\t")
line=paste("Rscript --vanilla t/", ANALYSIS, ".R ", paste(args, collapse = " "), sep="")
write(line,file=paste("finish_GWAS_", ANALYSIS, ".txt", sep=""),append=TRUE)
' > t/model_body.txt
sed -n 1,25p t/lnmxi.R > t/model_head.txt
cat t/model_head.txt t/model_body.txt > t/lnmxs.R

In [12]:
%%bash
# Create swarm for R analyses
rm -f swarm/GWAS_cohort_level_*.txt 
for i in $(tail -n +2 models.txt);do
    MODEL=$(echo $i | cut -d";" -f1)
    DATASET=$(echo $i | cut -d";" -f2)
    for INFILE in $(ls ../dataset/$DATASET/maf01rsq3_20Kcut/*);do
        echo "Rscript --vanilla t/$MODEL.R \"$i\" \"$INFILE\" \"outGWAS\"" >> swarm/GWAS_cohort_level_"$MODEL".txt 
    done
done
echo "
swarm -f swarm/GWAS_cohort_level_coxhm.txt --time=0:30:00 -g 2 -p 2 -b 18 --logdir swarm/t --module R --devel
swarm -f swarm/GWAS_cohort_level_lnmxi.txt --time=2:00:00 -g 2 -p2 -b 12 --logdir swarm/t --module R --devel
swarm -f swarm/GWAS_cohort_level_lnmxs.txt --time=2:00:00 -g 2 -p2 -b 12 --logdir swarm/t --module R --devel"
rm swarm/GWAS_cohort_level_lgsbl.txt swarm/GWAS_cohort_level_lnsgl.txt swarm/GWAS_cohort_level_lncns.txt # no analysis for those.


swarm -f swarm/GWAS_cohort_level_coxhm.txt --time=0:30:00 -g 2 -p 2 -b 18 --logdir swarm/t --module R --devel
swarm -f swarm/GWAS_cohort_level_lnmxi.txt --time=2:00:00 -g 2 -p2 -b 12 --logdir swarm/t --module R --devel
swarm -f swarm/GWAS_cohort_level_lnmxs.txt --time=2:00:00 -g 2 -p2 -b 12 --logdir swarm/t --module R --devel


In [13]:
%%bash
rm -rf outGWAS/*/lnmxs*

In [48]:
%%bash
swarm -f swarm/GWAS_cohort_level_coxhm.txt --time=0:30:00 -g 2 -p 2 -b 20 --logdir swarm/t --module R --devel
swarm -f swarm/GWAS_cohort_level_lnmxs.txt --time=2:00:00 -g 2 -b 19 --logdir swarm/t --module R --devel

Loading modules R
15931 commands run in 399 subjobs, each command requiring 2 gb and 1 thread, packing 2 processes per subjob, running 20 processes serially per subjob
sbatch --array=0-398 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=4096 --partition=norm --time=10:00:00 /spin1/swarm/iwakih2/uNrC9DyyUu/swarm.batch
Loading modules R
18540 commands run in 976 subjobs, each command requiring 2 gb and 1 thread, running 19 processes serially per subjob
sbatch --array=0-975 --job-name=swarm --output=swarm/t/swarm_%A_%a.o --error=swarm/t/swarm_%A_%a.e --cpus-per-task=1 --mem=2048 --partition=norm --time=1-14:00:00 /spin1/swarm/iwakih2/XecwudalC9/swarm.batch


In [3]:
%%bash
# sort -R swarm/GWAS_cohort_level_coxhm.txt > swarm/rGWAS_cohort_level_coxhm.txt
# swarm -f swarm/rGWAS_cohort_level_coxhm.txt --time=0:60:00 -g 2 -p 2 -b 20 --logdir swarm/t --module R --devel # 30min -> 60min 9717371
sort -R swarm/GWAS_cohort_level_lnmxs.txt > swarm/rGWAS_cohort_level_lnmxs.txt
swarm -f swarm/rGWAS_cohort_level_lnmxs.txt --time=2:30:00 -g 2 -p 2 -b 10 --logdir swarm/t --module R --devel 

Loading modules R
18539 commands run in 927 subjobs, each command requiring 2 gb and 1 thread, packing 2 processes per subjob, running 10 processes serially per subjob
sbatch --array=0-926 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=4096 --partition=norm --time=1-01:00:00 /spin1/swarm/iwakih2/0E4UtrVDYb/swarm.batch


In [1]:
%%bash
sed -e 's/ /" "/g' -e 's/S$/S"/' -e 's/t" "--vanilla" "t\/lnmxs.R"/t --vanilla t\/lnmxs.R/' finish_GWAS_lnmxs.txt > finish_GWAS_lnmxs2.txt
cat finish_GWAS_lnmxs2.txt swarm/GWAS_cohort_level_lnmxs.txt | sort | uniq -u | sort -R > finishNOT_GWAS_lnmxs.txt
swarm -f finishNOT_GWAS_lnmxs.txt --time=2:30:00 -g 2 -p 2 -b 10 --logdir swarm/t --module R --devel

Loading modules R
12 commands run in 1 subjob, each command requiring 2 gb and 1 thread, packing 2 processes per subjob, running 10 processes serially per subjob
sbatch --array=0-0 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=4096 --partition=norm --time=15:00:00 /spin1/swarm/iwakih2/AARTuIfMjs/swarm.batch


In [9]:
%%bash
# populate all the R results into one file (all.tab) Time consuming process. Filter abs(beta) < 10
# Join ALT_Frq and Rsq file. But because cut*.txt.gz includes multiple allele for filtered positions, remove A1 is not the one with MAF>0.01
rm -f swarm/populate*.txt
for RESULT in "outGWAS";do  
    for MODEL in coxhm lnmxi lnmxs;do
        for i in $(grep ^"$MODEL" models.txt);do
            DATASET=$(echo $i | cut -d";" -f2)
            OUTCOME=$(echo $i | cut -d";" -f3)
            if [[ "$OUTCOME" =~ ^(MMSE|MOCA|SEADL)$ ]]; then
                B_THRES=10
            else
                B_THRES=5
            fi
            echo "
cat $RESULT/$DATASET/$MODEL/$OUTCOME/*.txt | sed 's/_/\t/g' | cut -f 1-3,7-9 |\
 awk 'BEGIN{OFS=\"\t\"}{print \$1,\$3,\$2,\$4,\$5,\$6}' | awk 'NR==1{print;next} -$B_THRES < \$4 && \$4 < $B_THRES{print|\"LANG=C sort\"}' |\
 LANG=C join -t$'\t' --header - ../dataset/$DATASET/maf01rsq3_sorted.txt|\
 awk 'NR==1{print;next}\$2==\$8{print}' | cut -f 1-6,9-10 > $RESULT/$DATASET/$MODEL/$OUTCOME/all.tab" \
 >> swarm/populate_"$RESULT".txt
        done
    done
done

sed 's/lnmxi/lnmxs_intcp/g' swarm/populate_outGWAS.txt > test.txt # lnmxi -> lnmxs
head -n2 test.txt
swarm -f test.txt --time=0:01:00 -g 2 -b 40 --logdir swarm/t --partition=quick --devel

## result check
# ls outGWAS/*/lgsbl/*/all.tab > test.txt
# awk 'BEGIN{FS="/";OFS=";"}{print $3,$2,$4}'  test.txt > test2.txt
# cut -d';' -f1-3 models.txt | grep "lgsbl" > test3.txt
# cat test2.txt test3.txt | sort | uniq -c 

221 commands run in 3 subjobs, each command requiring 2 gb and 1 thread, running 82 processes serially per subjob
sbatch --array=0-2 --job-name=swarm --output=swarm/t/swarm_%A_%a.o --error=swarm/t/swarm_%A_%a.e --cpus-per-task=1 --mem=2048 --partition=quick --time=01:22:00 /spin1/swarm/iwakih2/EW13narttK/swarm.batch


In [11]:
%%bash
## lnsgl -> lnmxs_intcp (only 1 model == lnsgl;PARKFIT;MMSE;FEMALE,AAO,YEARfDIAG)
for i in $(ls */*/lnsgl/*/all.tab);do
    NEWDIR=$(echo $i | sed 's/lnsgl/lnmxs_intcp/' | sed 's/\/all.tab//')
    mkdir -p $NEWDIR
    cp $i $NEWDIR
    echo "$i -> $NEWDIR"
done

outGWAS/PARKFIT/lnsgl/MMSE/all.tab -> outGWAS/PARKFIT/lnmxs_intcp/MMSE
output/PARKFIT/lnsgl/MMSE/all.tab -> output/PARKFIT/lnmxs_intcp/MMSE


In [4]:
%%bash
# MetaAnalysis by METAL.
## First create files ready for meta analysis
# Logistic regression at baseline
RESULT="outGWAS"
rm -f meta/$RESULT/*/*.metal
for i in $(tail -n +2 models.txt);do
    MODEL=$(echo $i | cut -d";" -f1)
    DATASET=$(echo $i | cut -d";" -f2)
    OUTCOME=$(echo $i | cut -d";" -f3)
    if [ $MODEL == "lnmxi" ];then # lnsgl will be combined with lnmxi
        MODEL=lnmxs_intcp
    elif [ $MODEL == "lnsgl" ];then # lnsgl will be combined with lnmxi
        MODEL=lnmxs_intcp
    fi
    mkdir -p meta/$RESULT/$MODEL
    echo "
MARKER POS
ALLELE A1 A2
FREQ   ALT_Frq
EFFECT BETA
PVALUE P
STDERR SE
GENOMICCONTROL ON 
PROCESS $RESULT/$DATASET/$MODEL/$OUTCOME/all.tab" >> meta/$RESULT/$MODEL/$OUTCOME.metal
done
for i in $(ls meta/$RESULT/*/*.metal);do
    sed -i -e '1 iSCHEME STDERR\nAVERAGEFREQ ON\nMINMAXFREQ ON' \
    -e "\$aOUTFILE $(echo $i | cut -d"." -f1) .tbl\nANALYZE HETEROGENEITY\nQUIT" $i
done

In [6]:
%%bash
rm -f swarm/metal.txt
for i in $(ls meta/outGWAS/*/*.metal);do
    echo "metal $i" >> swarm/metal.txt
done
rm -rf swarm/metal
grep 'coxhm\|lnmxs\|lnmxs_intcp\|lgsbl' swarm/metal.txt > swarm/metal_outGWAS.txt
swarm -f swarm/metal_outGWAS.txt -g 6 -p 2 -b 3 --time=1:20:00 --module metal --logdir swarm/metal --partition=quick --devel 

Loading modules metal
42 commands run in 7 subjobs, each command requiring 6 gb and 1 thread, packing 2 processes per subjob, running 3 processes serially per subjob
sbatch --array=0-6 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=12288 --partition=quick --time=04:00:00 /spin1/swarm/iwakih2/2ijg971BkT/swarm.batch


In [7]:
%%bash
swarm -f swarm/metal_outGWAS.txt -g 6 -p 2 -b 3 --time=0:30:00 --module metal --logdir swarm/metal --partition=quick --devel

10759074


In [10]:
%%bash
# Rscript for MH plot QQ plot
echo '
args <- commandArgs(trailingOnly = TRUE)
require("qqman");require("data.table");require("tidyr");require(dplyr)
FOLDER=args[1]
RESULT=args[2]
MODEL=args[3]
OUTCOME=args[4]
MH = fread(cmd = paste("cut -f 1,10,16 ",FOLDER, "/", RESULT, "/", MODEL, "/", OUTCOME, "_filt.txt", sep="")) %>%
  separate(MarkerName, c("CHR", "BP"), sep=":") %>% rename(P=`P-value`, SNP=ID) %>% 
  mutate_at(vars("CHR", "BP", "P"), as.numeric)
# lambda
chisq <- qchisq(1-median(MH$P),1)
lambda = median(chisq)/qchisq(0.5,1)
# QQ plot
png(paste(FOLDER, "/", RESULT, "/", MODEL, "/", OUTCOME, "_QQ.png", sep=""))
qq(MH$P, main = paste(OUTCOME, "in", MODEL))
text(4, 1, sprintf("lambda = %.3f", lambda))
text(4, 2, paste("N of variants =", nrow(MH)))
dev.off()
# MH plot if lambda is not too large.
if(lambda < 1.1){
    maxlogP=-log10(min(MH$P))
    png(paste(FOLDER, "/", RESULT, "/", MODEL, "/", OUTCOME,"_MH.png", sep=""), width=2000, height=1000, pointsize=18)
    manhattan(MH, col = c("blue4", "orange3"), main=paste(OUTCOME, "in", MODEL), cex.axis = 0.6, annotatePval = 0.00001, ylim=c(0, max(10, maxlogP)))
    dev.off()
}' > t/QQMH.R



# count the number of datasets in the meta-analysis ...1hour. 
## filter only results reported at least half of the cohorts, and I2<80.    FreqMax - FreqMix<0.15 
## MAF 0.05
cut -d";" -f 1,3 models.txt | sort | uniq -c |sed 's/;/ /g'| awk 'BEGIN{OFS=","}{print $2,$3,$1}' > t/NinMeta.csv
FOLDER=meta
RESULT=outGWAS
rm -f swarm/"$FOLDER"_filt.txt
rm -f $FOLDER/$RESULT/GWAS_signal.txt
for MODEL in coxhm lgsbl lnmxi lnmxs;do
    for i in $(grep ^"$MODEL" t/NinMeta.csv);do
        OUTCOME=$(echo $i | cut -d, -f2)
        N_COHORTs=$(echo $i | cut -d, -f3)
        THRES=$(echo $[(N_COHORTs) / 2 ])
        if [ $MODEL == lnmxi ];then
            MODEL=lnmxs_intcp # lnmxi -> lnmxs_intcp
        fi
        echo "
awk 'NR==1{print;next}\$14>='$THRES' && \$14 > 0 && \$12<80 && \$7-\$6 < 0.15 && \$4>0.05  && \$4<0.95{print| \"LANG=C sort\"}' $FOLDER/$RESULT/$MODEL/"$OUTCOME"1.tbl |\
 join -t\$'\t' --header - ../tool/HRC_RS_conversion_sorted.txt > $FOLDER/$RESULT/$MODEL/"$OUTCOME"_filt.txt;\
Rscript --vanilla t/QQMH.R $FOLDER $RESULT $MODEL $OUTCOME;\
echo "$MODEL $OUTCOME" >> $FOLDER/$RESULT/GWAS_signal.txt;\
awk '\$10+0 < 5e-8 {print}' $FOLDER/$RESULT/$MODEL/"$OUTCOME"_filt.txt >> $FOLDER/$RESULT/GWAS_signal.txt" >> swarm/"$FOLDER"_filt.txt
    done
done
echo swarm -f swarm/"$FOLDER"_filt.txt -g 6 -p 2 -b 3 --time=0:10:00 --module R --logdir swarm/metal --partition=quick --devel
swarm -f swarm/meta_filt.txt -g 6 -p 2 -b 3 --time=0:10:00 --module R --logdir swarm/metal --partition=quick --devel

swarm -f swarm/meta_filt.txt -g 6 -p 2 -b 3 --time=0:10:00 --module R --logdir swarm/metal --partition=quick --devel
Loading modules R
42 commands run in 7 subjobs, each command requiring 6 gb and 1 thread, packing 2 processes per subjob, running 3 processes serially per subjob
sbatch --array=0-6 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=12288 --partition=quick --time=30:00 /spin1/swarm/iwakih2/7WUCYVR0lg/swarm.batch


In [13]:
%%bash
rm -rf meta/outGWAS/*/*.png
swarm -f swarm/meta_filt.txt -g 6 -p 2 -b 3 --time=0:10:00 --module R --logdir swarm/metal --partition=quick --dependency=afterany:10759074 --devel

10759185


In [ ]:
%%bash
awk 'NR==1{print;next}$14>='4' && $14 > 0 && $12<80 && $7-$6 < 0.15 && $4>0.05  && $4<0.95{print| "LANG=C sort"}' meta/outGWAS/coxhm/HY31.tbl | join -t$'\t' --header - ../tool/HRC_RS_conversion_sorted.txt > meta/outGWAS/coxhm/HY3_filt.txt;Rscript --vanilla t/QQMH.R meta outGWAS coxhm HY3;echo coxhm HY3 >> meta/outGWAS/GWAS_signal.txt;awk '$10+0 < 5e-8 {print}' meta/outGWAS/coxhm/HY3_filt.txt >> meta/outGWAS/GWAS_signal.txt

In [1]:
%%bash
grep '9:108058562' outGWAS/*/coxhm/HY3/all.tab > t/forest.txt

In [3]:
%%bash
head -n1 outGWAS/CORIELL//coxhm/HY3/all.tab

POS	A1	A2	BETA	SE	P	ALT_Frq	Rsq


# Rsq>0.8 MAF>0.05

In [9]:
%%bash
# populate all the R results into one file (all.tab) Time consuming process. Filter abs(beta) < 10
# Join ALT_Frq and Rsq file. But because cut*.txt.gz includes multiple allele for filtered positions, remove A1 is not the one with MAF>0.01
rm -f swarm/populate*.txt
for RESULT in "outGWASrsq8";do  
    for MODEL in coxhm lnmxi lnmxs;do
        for i in $(grep ^"$MODEL" models.txt);do
            DATASET=$(echo $i | cut -d";" -f2)
            OUTCOME=$(echo $i | cut -d";" -f3)
            if [ $MODEL == "lnmxi" ];then # lnsgl will be combined with lnmxi
                MODEL=lnmxs_intcp
            fi
            mkdir -p $RESULT/$DATASET/$MODEL/$OUTCOME
echo "awk '\$8>0.8{print}' outGWAS/$DATASET/$MODEL/$OUTCOME/all.tab > $RESULT/$DATASET/$MODEL/$OUTCOME/all.tab" >> swarm/populate_"$RESULT".txt
        done
    done
done

grep "lnmxs_intcp" swarm/populate_"$RESULT".txt > test.txt # lnmxi -> lnmxs
swarm -f test.txt --time=0:01:00 -g 2 -b 40 --logdir swarm/t --partition=quick --devel

74 commands run in 2 subjobs, each command requiring 2 gb and 1 thread, running 40 processes serially per subjob
sbatch --array=0-1 --job-name=swarm --output=swarm/t/swarm_%A_%a.o --error=swarm/t/swarm_%A_%a.e --cpus-per-task=1 --mem=2048 --partition=quick --time=40:00 /spin1/swarm/iwakih2/LS2kngqE0I/swarm.batch


In [4]:
%%bash
mkdir outGWASrsq8/PARKFIT/lnmxs_slope/MMSE
cp outGWAS/PARKFIT/lnsgl/MMSE/all.tab outGWASrsq8/PARKFIT/lnmxs_slope/MMSE

In [11]:
%%bash
RESULT="outGWASrsq8"
rm -f meta/$RESULT/*/*.metal
for i in $(tail -n +2 models.txt);do
    MODEL=$(echo $i | cut -d";" -f1)
    DATASET=$(echo $i | cut -d";" -f2)
    OUTCOME=$(echo $i | cut -d";" -f3)
    if [ $MODEL == "lnmxi" ];then # lnsgl will be combined with lnmxi
        MODEL=lnmxs_intcp
    elif [ $MODEL == "lnsgl" ];then # lnsgl will be combined with lnmxi
        MODEL=lnmxs_intcp
    fi
    mkdir -p meta/$RESULT/$MODEL
    echo "
MARKER POS
ALLELE A1 A2
FREQ   ALT_Frq
EFFECT BETA
PVALUE P
STDERR SE
PROCESS $RESULT/$DATASET/$MODEL/$OUTCOME/all.tab" >> meta/$RESULT/$MODEL/$OUTCOME.metal
done

for i in $(ls meta/$RESULT/*/*.metal);do
    sed -i -e '1 iSCHEME STDERR\nAVERAGEFREQ ON\nMINMAXFREQ ON' \
    -e "\$aOUTFILE $(echo $i | cut -d"." -f1) .tbl\nANALYZE HETEROGENEITY\nQUIT" $i
done

rm -f swarm/metal.txt
for i in $(ls meta/$RESULT/*/*.metal);do
    echo "metal $i" >> swarm/metal.txt
done
rm -rf swarm/metal
grep 'coxhm\|lnmxs' swarm/metal.txt > swarm/metal_outGWAS.txt
swarm -f swarm/metal_outGWAS.txt -g 6 -p 2 -b 3 --time=1:20:00 --module metal --logdir swarm/metal --partition=quick --devel 

Loading modules metal
30 commands run in 5 subjobs, each command requiring 6 gb and 1 thread, packing 2 processes per subjob, running 3 processes serially per subjob
sbatch --array=0-4 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=12288 --partition=quick --time=04:00:00 /spin1/swarm/iwakih2/CGJcPfffSf/swarm.batch


In [21]:
%%bash
echo '
args <- commandArgs(trailingOnly = TRUE)
require("qqman");require("data.table");require("tidyr");require(dplyr)
FOLDER=args[1]
RESULT=args[2]
MODEL=args[3]
OUTCOME=args[4]
MH = fread(cmd = paste("cut -f 1,10,16 ",FOLDER, "/", RESULT, "/", MODEL, "/", OUTCOME, "_filt.txt", sep="")) %>%
  separate(MarkerName, c("CHR", "BP"), sep=":") %>% rename(P=`P-value`, SNP=ID) %>% 
  mutate_at(vars("CHR", "BP", "P"), as.numeric)
# lambda
chisq <- qchisq(1-median(MH$P),1)
lambda = median(chisq)/qchisq(0.5,1)
# QQ plot
png(paste(FOLDER, "/", RESULT, "/", MODEL, "/", OUTCOME, "_QQ.png", sep=""))
qq(MH$P, main = paste(OUTCOME, "in", MODEL))
text(4, 1, sprintf("lambda = %.3f", lambda))
text(4, 2, paste("N of variants =", nrow(MH)))
dev.off()
# MH plot if lambda is not too large.
if(lambda < 1.1){
    maxlogP=-log10(min(MH$P))
    png(paste(FOLDER, "/", RESULT, "/", MODEL, "/", OUTCOME,"_MH.png", sep=""), width=2000, height=1000, pointsize=18)
    manhattan(MH, col = c("blue4", "orange3"), main=paste(OUTCOME, "in", MODEL), cex.axis = 0.6, annotatePval = 0.00001, ylim=c(0, max(10, maxlogP)))
    dev.off()
}' > t/QQMH.R



# count the number of datasets in the meta-analysis ...1hour. 
## filter only results reported at least half of the cohorts, and I2<80.    FreqMax - FreqMix<0.15 
## MAF 0.05
cut -d";" -f 1,3 models.txt | sort | uniq -c |sed 's/;/ /g'| awk 'BEGIN{OFS=","}{print $2,$3,$1}' > t/NinMeta.csv
FOLDER=meta
RESULT=outGWASrsq8
rm -f swarm/"$FOLDER"_filt.txt
rm -f $FOLDER/$RESULT/GWAS_signal.txt
for MODEL in coxhm lgsbl lnmxi lnmxs;do
    for i in $(grep ^"$MODEL" t/NinMeta.csv);do
        OUTCOME=$(echo $i | cut -d, -f2)
        N_COHORTs=$(echo $i | cut -d, -f3)
        THRES=$(echo $[(N_COHORTs) / 2 ])
        if [ $MODEL == lnmxi ];then
            MODEL=lnmxs_intcp # lnmxi -> lnmxs_intcp
        fi
        echo "
awk 'NR==1{print;next}\$14>=$THRES && \$14 > 0 && \$12<80 && \$7-\$6 < 0.15 && \$4>0.05  && \$4<0.95{print| \"LANG=C sort\"}' $FOLDER/$RESULT/$MODEL/"$OUTCOME"1.tbl |\
 join -t\$'\t' --header - ../tool/HRC_RS_conversion_sorted.txt > $FOLDER/$RESULT/$MODEL/"$OUTCOME"_filt.txt;\
Rscript --vanilla t/QQMH.R $FOLDER $RESULT $MODEL $OUTCOME;\
echo "$MODEL $OUTCOME" >> $FOLDER/$RESULT/GWAS_signal.txt;\
awk '\$10+0 < 5e-8 {print}' $FOLDER/$RESULT/$MODEL/"$OUTCOME"_filt.txt >> $FOLDER/$RESULT/GWAS_signal.txt" >> swarm/"$FOLDER"_filt.txt
    done
done
swarm -f swarm/"$FOLDER"_filt.txt -g 6 -p 2 -b 7 --time=0:10:00 --module R --logdir swarm/metal --partition=quick --devel
echo "swarm -f swarm/"$FOLDER"_filt.txt -g 6 -p 2 -b 7 --time=0:10:00 --module R --logdir swarm/metal --partition=quick"

Loading modules R
42 commands run in 3 subjobs, each command requiring 6 gb and 1 thread, packing 2 processes per subjob, running 7 processes serially per subjob
sbatch --array=0-2 --job-name=swarm --output=/dev/null --error=/dev/null --cpus-per-task=2 --mem=12288 --partition=quick --time=01:10:00 /spin1/swarm/iwakih2/msB__naEmO/swarm.batch
swarm -f swarm/meta_filt.txt -g 6 -p 2 -b 7 --time=0:10:00 --module R --logdir swarm/metal --partition=quick


Rsq > 0.8 severly restrict the number of SNPs to test.    

# Why results are severely distorted? Check individual cohort

MH = ... %>% filter(Rsq>0.8) %>% ...: can be adjustable
MH = ... %>% filter(ALT_Frq>0.1) %>% ...: can be adjustable

In [28]:
%%bash
echo '
args <- commandArgs(trailingOnly = TRUE)
require("qqman");require("data.table");require("tidyr");require(dplyr)
RESULT=args[1]
DATASET=args[2]
MODEL=args[3]
OUTCOME=args[4]
MH = fread(cmd = paste("cut -f 1,6,7,8 ", RESULT, "/", DATASET, "/", MODEL, "/", OUTCOME, "/", "all.tab", sep="")) %>%
  separate(POS, c("CHR", "BP"), sep=":") %>% filter(ALT_Frq>0.1) %>%
  mutate_at(vars("CHR", "BP", "P"), as.numeric)
# lambda
chisq <- qchisq(1-median(MH$P),1)
lambda = median(chisq)/qchisq(0.5,1)
# QQ plot
png(paste(RESULT, DATASET, MODEL, OUTCOME, "aQQ.png", sep="/"))
qq(MH$P, main = paste(OUTCOME, "in", MODEL))
text(4, 1, sprintf("lambda = %.3f", lambda))
text(4, 2, paste("N of variants =", nrow(MH)))
dev.off()
' > t/QQMH.R



RESULT=outGWAS
DATASET=PPMI
MODEL=coxhm
OUTCOME=HY3
echo "
Rscript --vanilla t/QQMH.R $RESULT $DATASET $MODEL $OUTCOME"


Rscript --vanilla t/QQMH.R outGWAS PPMI coxhm HY3
